In [1]:
!pip install chardet 

In [2]:
def f_get_enc_type(src_path):
    '''
    encoding type 반환
    utf-8, euk-kr 등등
    '''
    import chardet
    
    rawdata = open(src_path, "rb").read()
    result = chardet.detect(rawdata)
    enc = result['encoding']
    
    return enc
    

In [3]:
def f_get_dir_list(src_dir):
    '''
    src_dir 에서 디렉토리만 추출해서 리스트로 반환
    '''
    item_list = os.listdir(src_dir)
    
    dir_list = []
    for item in item_list:
        path = os.path.join(src_dir, item)
        
        if os.path.isdir(path):
            dir_list.append(item)
            
    return dir_list

In [4]:
def f_get_file_list(src_dir, ext='.'):
    '''
    src_dir 에서 파일명을 추출해서 리스트로 반환
    f_get_file_list(src_dir, 'xml') >> xml 파일만 리스트로 반환
    '''
    item_list = os.listdir(src_dir)
    
    file_list = []
    for item in item_list:
        path = os.path.join(src_dir, item)
        
        if (os.path.isfile(path)) and (ext in item):
            file_list.append(item)
            
    return file_list

In [5]:
def f_file_to_enc_utf8(src_path):
    '''
    입력 받은 파일이 utf-8이 아니면,
    파일을 읽어서 같은 파일 이름의 utf-8 로 저장
    '''
    if f_get_enc_type(src_path) != 'utf-8':
        with open(src_path, 'r') as f:
            lines = f.readlines()

        with open(src_path, 'w', encoding='utf-8') as f:
            f.writelines(lines)

In [6]:
def f_xml_path_cut_hangule(xml_path):
    '''
    ./data/name.xml 입력을 받아
    <path>태그의 내용 중 한글을 제거하고
    xml 파일을 저장한다.
    '''
    import os
    import re
    
    # 경로, 파일명 분리
    xml_file = os.path.split(xml_path)[1]
    
    # xml 파일이 아니면 return
    if os.path.splitext(xml_file)[1] != '.xml':
        print('xml 파일 아님:', xml_file)
        return
    
    # 파일명에서 한글 제거
    regex = '[()가-힣]'  # 한글 ( ) 제외
    ss = re.sub(regex, '', xml_file)
    
    # 파일명에서 약번호 가져오기
    if 'after' in xml_file:
        alyac_num = xml_file.split('_')[1].split('.')[0]
    else:
        alyac_num = xml_file.split('.')[0]
    
    # xml 파일 읽기
    with open(xml_path, mode='r', encoding='utf-8') as f:
        lines = f.readlines()
#         print(lines)
    
    # xml 파일 내용 갱신
    for i in range(len(lines)):
        # 스페이스 4칸을 탭으로 변경
        if '    ' in lines[i]:
            lines[i] = lines[i].replace('    ', '\t')
            
        # <path> 태그 내용을 한글 제거한 파일명으로 대체
        if '<path>' in lines[i]:
            lines[i] = '\t<path>'+ ss +'</path>\n'
            
        # <name> 태그 내용을 약 번호로 대체
        if '<name>' in lines[i]:
            lines[i] = '\t\t<name>'+ alyac_num +'</name>\n'
            
        # <width> or <height> 가 150 이 아니면 150 으로 수정
        if (('<width>' in lines[i]) or ('<height>' in lines[i])) and ('151' in lines[i]):
            lines[i] = lines[i].replace('151', '150')
        
        if '<xmin>' in lines[i]:
            xmin = lines[i].replace('<xmin>', '').replace('</xmin>', '')
            if int(xmin) < 0:
                print('xmin:', xmin, xml_file)
            
        if '<ymin>' in lines[i]:
            ymin = lines[i].replace('<ymin>', '').replace('</ymin>', '')
            if int(ymin) < 0:
                print('ymin:', ymin, xml_file)
                
        if '<xmax>' in lines[i]:
            xmax = lines[i].replace('<xmax>', '').replace('</xmax>', '')
            if int(xmax) < 0:
                print('xmax:', xmax, xml_file)
                
        if '<ymax>' in lines[i]:
            ymax = lines[i].replace('<ymax>', '').replace('</ymax>', '')
            if int(ymax) < 0:
                print('ymax:', ymax, xml_file)
            
    
    # xml 파일 쓰기
    with open(xml_path, mode='w', encoding='utf-8') as f:
        f.writelines(lines)
        
#     print(xml_path)

In [7]:
import os

src_root_dir = './alyac_img_label/google_drive'

dir_1st_list = f_get_dir_list(src_root_dir)

for i, dir_1st in enumerate(dir_1st_list):
    dir_1st_dir = os.path.join(src_root_dir, dir_1st)
    print(f'{i}: {dir_1st}')
    
    # 1st 디렉토리 안에 디렉토리 정보 확인
    dir_2nd_list = f_get_dir_list(dir_1st_dir)
    
    # 2nd 디렉토리가 있으면
    if len(dir_2nd_list) > 0:
        for dir_2nd in dir_2nd_list:
            dir_2nd_dir = os.path.join(dir_1st_dir, dir_2nd)
            dir_2nd_file_list = f_get_file_list(dir_2nd_dir, 'xml')
            
            # 2nd 디렉토리 안에 xml 파일 수정
            for dir_2nd_file in dir_2nd_file_list:
                dir_2nd_path = os.path.join(dir_2nd_dir, dir_2nd_file)
                
                f_file_to_enc_utf8(dir_2nd_path)
                f_xml_path_cut_hangule(dir_2nd_path)
        
    # 1st 디렉토리 안에 xml 파일 리스트 추출
    dir_1st_file_list = f_get_file_list(dir_1st_dir, 'xml')
    for dir_1st_file in dir_1st_file_list:
        dir_1st_path = os.path.join(dir_1st_dir, dir_1st_file)
        
        f_file_to_enc_utf8(dir_1st_path)
        f_xml_path_cut_hangule(dir_1st_path)
        
print('완료')

0: 12063.한미오메가연질캡슐1000밀리그램_black
1: 12063.한미오메가연질캡슐1000밀리그램_crop
2: 17159.제스부틴정_black
3: 17159.제스부틴정_crop
4: 17458.캐롤비콜드연질캡슐_black
5: 17458.캐롤비콜드연질캡슐_crop
6: 2070.탁센연질캡슐(나프록센)_black
7: 2070.탁센연질캡슐(나프록센)_crop
8: 22074.모드콜에스연질캡슐_black
9: 22074.모드콜에스연질캡슐_crop
10: 22408.이지엔6프로연질캡슐_black
11: 22408.이지엔6프로연질캡슐_crop
12: 22605.타이레놀정500밀리그람_black
13: 22605.타이레놀정500밀리그람_crop
14: 3324.로프민캡슐_black
15: 3324.로프민캡슐_crop
16: 3982.애드빌정(이부프로펜)_black
17: 3982.애드빌정(이부프로펜)_crop
18: 8998.이소티논연질캡슐10밀리그램_black
19: 8998.이소티논연질캡슐10밀리그램_crop
완료
